#ETL dataset `users_items.gz.json`

En este notebook, nuestro objetivo es realizar el proceso de extracción, transformación y carga (ETL) de los datos que contienen información sobre los usuarios y los juegos que consumen en Steam, para poder disponer de ellos mediante una API. Este proceso nos permitirá acceder a la información del contenido de los usuarios de forma estructurada y actualizada, así como preparar los datos para su posterior análisis y modelado.

## 0 Configuraciones Globales e Importaciones

En esta sección, importamos todas las bibliotecas y modulos necesarios para nuestro proceso ETL y establecemos configuraciones globales de ser requerido.

In [ ]:
import sys
import pandas as pd
import gzip
import ast

print(f"System version: {sys.version}")
print(f"Pandas version: {pd.__version__}")

System version: 3.10.12 (main, Jun 11 2023, 05:26:28) [GCC 11.4.0]
Pandas version: 1.5.3


## 1 Extracción

En esta sección, extraemos los datos del archivo`users_items.gz.json` y describimos a detalle su contenido.

### 1.1 Extracción de los datos

En este caso, el conjunto de datos no cumple con un formato válido de JSON, donde el par clave-valor deben estar en comillas dobles. Por lo tanto, se optó por usar `ast.literal_eval`. Esta función es útil para analizar cadenas JSON que tienen comillas simples en lugar de dobles.

Los datos se extraen descomprimiendo el archivo con el módulo `gzip`, se recorre cada línea del archivo y se interpreta como una estructura de datos de `Python` usando el modulo `ast`. Se almacenan en una lista y se cargan a un Dataframe de `pandas` para observar su contenido.

In [ ]:
# Ruta al dataset
path = '/content/drive/MyDrive/data/raw/users_items.json.gz'

# Leemos el archivo usando ast.literal_eval para analizar la cadena JSON
data = []
with gzip.open(path, 'r') as f:
    for line in f:
        data.append(ast.literal_eval(line.decode("utf-8")))

# Convertimos a DataFrame
df_items = pd.DataFrame(data)
df_items.head()

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


- Hacemos un resumen conciso del Dataframe para observar los tipos de datos por columnas, verificar nulos.

In [ ]:
df_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88310 entries, 0 to 88309
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      88310 non-null  object
 1   items_count  88310 non-null  int64 
 2   steam_id     88310 non-null  object
 3   user_url     88310 non-null  object
 4   items        88310 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.4+ MB


- Observamos que la estructura de los datos de la columna 'items' parece contener una lista de diccionarios.

In [ ]:
df_items['items'][0][:3]

[{'item_id': '10',
  'item_name': 'Counter-Strike',
  'playtime_forever': 6,
  'playtime_2weeks': 0},
 {'item_id': '20',
  'item_name': 'Team Fortress Classic',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '30',
  'item_name': 'Day of Defeat',
  'playtime_forever': 7,
  'playtime_2weeks': 0}]

- Comprobamos que efectivamente es una lista de diccionarios y procedemos a desanidarlo.

In [ ]:
meta = df_items.columns[:4].tolist()
df_items_expanded = pd.json_normalize(data, 'items', meta)
df_items_expanded.head()

,item_id,item_name,playtime_forever,playtime_2weeks,user_id,items_count,steam_id,user_url
0,10,Counter-Strike,6,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
1,20,Team Fortress Classic,0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
2,30,Day of Defeat,7,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
3,40,Deathmatch Classic,0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
4,50,Half-Life: Opposing Force,0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...


* Comprobamos datos del df desanidado.

In [ ]:
df_items_expanded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5153209 entries, 0 to 5153208
Data columns (total 8 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   item_id           object
 1   item_name         object
 2   playtime_forever  int64 
 3   playtime_2weeks   int64 
 4   user_id           object
 5   items_count       object
 6   steam_id          object
 7   user_url          object
dtypes: int64(2), object(6)
memory usage: 314.5+ MB


### 1.2 Descripción del los datos.

A partir del análisis exploratorio anterior, podemos observar que el conjunto de datos contiene 5153209 filas y 8 columnas con información de los usuarios de Steam y el contenido que consumen. A continuación, se describe el contenido de las variables:

- **user_id**: identificador único del usuario.
- **items_count**: número total de juegos que posee el usuario
- **steam_id**: identificador único SteamID de la cuenta.
- **user_url**: URL del perfil del usuario.
- **items**: items del usuario en formato JSON. Contiene las siguientes claves:
    - **item_id**: identificador único del juego.
    - **item_name**: nombre del juego.
    - **playtime_forever**:  número total de minutos jugados desde que se tiene registro.
    - **playtime_2weeks**: número total de minutos jugados en las últimas 2 semanas.



## 2 Transformación

En esta sección, realizamos la limpieza inicial de los datos y las transformaciones necesarias. Esto puede incluir la creación de nuevas columnas a partir de las existentes, la eliminación de duplicados o columnas innecesarias, la gestión de valores nulos o la corrección de tipos de datos.

### 2.1 Gestión de valores nulos

Comprobamos nulos:

In [ ]:
df_items_expanded.isnull().sum()

item_id             0
item_name           0
playtime_forever    0
playtime_2weeks     0
user_id             0
items_count         0
steam_id            0
user_url            0
dtype: int64

No tenemos nulos por lo que ahora comprobamos duplicados.

### 2.2 Verificación de duplicados

Comprobamos si tenemos duplicados:

In [ ]:
df_items_expanded.duplicated().sum()

59104

Tenemos duplicados que procedemos a eliminar.

In [ ]:
df_items_expanded.drop_duplicates(inplace=True)
df_items_expanded.shape

(5094105, 8)

### 2.3 Conversión de minutos a horas en `playtime_forever`

Como queremos que nuestra API nos devuelva el tiempo de juego en horas y no en minutos, convertimos la columna 'playtime_forever' a la unidad adecuada. Así, facilitamos el análisis de los datos y evitamos posibles errores de interpretación.

In [ ]:
df_items_expanded['playtime_forever'] = df_items_expanded['playtime_forever'] / 60
df_items_expanded.head()

,item_id,item_name,playtime_forever,playtime_2weeks,user_id,items_count,steam_id,user_url
0,10,Counter-Strike,0.100000,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
1,20,Team Fortress Classic,0.000000,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
2,30,Day of Defeat,0.116667,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
3,40,Deathmatch Classic,0.000000,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
4,50,Half-Life: Opposing Force,0.000000,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...


### 2.4 Eliminación de columnas innecesarias

Para optimizar nuestras consultas en la API, eliminamos las columnas 'playtime_2weeks', 'steam_id' y 'user_url', que no son necesarias para nuestro análisis. También eliminamos la columna 'item_name', que es redundante, y la columna 'items_count', que aunque podría facilitar el conteo de items, eliminarla ayuda a reducir el tamaño de nuestra base de datos. De esta manera, logramos una base de datos más pequeña y eficiente.



In [ ]:
columns = ['playtime_2weeks', 'steam_id', 'user_url', 'item_name']
df_items_expanded.drop(columns, axis=1, inplace=True)
df_items_expanded.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5094105 entries, 0 to 5153208
Data columns (total 4 columns):
 #   Column            Dtype  
---  ------            -----  
 0   item_id           object 
 1   playtime_forever  float64
 2   user_id           object 
 3   items_count       object 
dtypes: float64(1), object(3)
memory usage: 194.3+ MB


## 3 Carga

Finalmente, en esta sección cargamos nuestros datos transformados a un destino interino para su posterior análisis y tratamiento mediante feature engineering. Optamos por almacenarlos en formato parquet con compresion snappy para reducir su tamaño de almacenamiento.

In [ ]:
#Exportamos a parquet
path = 'data/interim/user_items.parquet'
df_items_expanded.to_parquet(path, engine='pyarrow', compression='snappy')
print(f'El archivo se guardó correctamente en {path}')

El archivo se guardó correctamente en data/interim/user_items.parquet


## 4 Referencias

* Steam store. (s/f). Steampowered.com. Recuperado el 25 de octubre de 2023, de https://store.steampowered.com/

* Steam web API. (s/f). Valvesoftware.com. Recuperado el 25 de octubre de 2023, de https://developer.valvesoftware.com/wiki/Steam_Web_API

* Steam community. (s/f). Steamcommunity.com. Recuperado el 25 de octubre de 2023, de https://steamcommunity.com/

